In [ ]:
#Class Labels: {'Vertical': 0, 'defect free': 1, 'hole': 2, 'horizontal': 3, 'lines': 4, 'stain': 5}

In [1]:
import tensorflow as tf

In [3]:
model=tf.keras.models.load_model("fabric_defect_model.h5")
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 86528)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │      11,075,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 6)                   │             774 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,169,736 (42.61 MB)

 Trainable params: 11,169,734 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [4]:
#code for checking image via path
import numpy as np
from tensorflow.keras.preprocessing import image

# Load an image for testing

img_path = r"C:\Users\viren\OneDrive\Pictures\Screenshots\Screenshot 2025-03-29 124647.png"  # Change to your image path
img = image.load_img(img_path, target_size=(224, 224))  # Resize to model's input size
img_array = image.img_to_array(img) / 255.0  # Normalize
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

# Predict
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions, axis=1)

print(f'Predicted Class: {predicted_class}')

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\viren\\OneDrive\\Pictures\\Screenshots\\Screenshot 2025-03-29 124647.png'

In [6]:
#code via popup
import sys
import numpy as np
from PIL import Image
import tensorflow as tf
from PyQt6.QtWidgets import (
    QApplication, QWidget, QLabel, QPushButton, QFileDialog,
    QMessageBox, QVBoxLayout, QHBoxLayout, QProgressBar
)
from PyQt6.QtGui import QPixmap
from PyQt6.QtCore import Qt, QTimer

# Load the model
model = tf.keras.models.load_model("fabric_defect_model.h5")

# Class label mapping
class_labels = {
    0: 'Vertical',
    1: 'defect free',
    2: 'hole',
    3: 'horizontal',
    4: 'lines',
    5: 'stain'
}

class ImagePredictor(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Fabric Defect Predictor")
        self.setGeometry(200, 200, 650, 400)

        # Layout
        layout = QVBoxLayout()

        # Upload button
        self.upload_button = QPushButton("Upload Image")
        self.upload_button.clicked.connect(self.upload_image)
        layout.addWidget(self.upload_button)

        # Image display
        self.image_label = QLabel("No image uploaded")
        self.image_label.setAlignment(Qt.AlignmentFlag.AlignCenter)
        self.image_label.setFixedSize(300, 300)
        layout.addWidget(self.image_label)

        # Progress bar as loader
        self.loader = QProgressBar()
        self.loader.setRange(0, 0)  # Indeterminate
        self.loader.setVisible(False)
        layout.addWidget(self.loader)

        # Prediction label
        self.result_label = QLabel("Prediction: -")
        self.result_label.setAlignment(Qt.AlignmentFlag.AlignCenter)
        layout.addWidget(self.result_label)

        self.setLayout(layout)

    def upload_image(self):
        file_path, _ = QFileDialog.getOpenFileName(self, "Open Image", "", "Image Files (*.png *.jpg *.jpeg)")
        if file_path:
            self.loader.setVisible(True)
            self.result_label.setText("Predicting...")
            self.display_image(file_path)

            # Delay prediction slightly to show loader
            QTimer.singleShot(100, lambda: self.predict_and_show(file_path))

    def display_image(self, path):
        pixmap = QPixmap(path).scaled(300, 300, Qt.AspectRatioMode.KeepAspectRatio)
        self.image_label.setPixmap(pixmap)

    def predict_and_show(self, path):
        try:
            img = Image.open(path).convert("RGB").resize((224, 224))
            img_array = np.array(img) / 255.0
            img_array = np.expand_dims(img_array, axis=0)

            predictions = model.predict(img_array)
            predicted_index = np.argmax(predictions, axis=1)[0]
            predicted_label = class_labels.get(predicted_index, "Unknown")

            self.result_label.setText(f"✅ Prediction: {predicted_label}")
        except Exception as e:
            QMessageBox.critical(self, "Error", f"Prediction failed:\n{str(e)}")
        finally:
            self.loader.setVisible(False)

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = ImagePredictor()
    window.show()
    sys.exit(app.exec())

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 284ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step


SystemExit: 0

C:\Users\viren\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
